In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from time import time
from numpy import array

df = pd.read_csv("train/captions.txt")

In [2]:
train, val = np.split(df.sample( frac=1,random_state=42), [int(.8*len(df)),])
print(len(df), train.shape, val.shape)

40455 (32364, 2) (8091, 2)


/home/dniit/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [3]:
image_path = 'train/Images/'

In [4]:
import re
def caption_preprocessing(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    # tokenize
    text=text.split()
    # convert to lower case
    text = [word.lower() for word in text]
    # remove hanging 's' and 'a'
    # text = [word for word in text if len(word)>1]

    # remove tokens with numbers in them
    text = [word for word in text if word.isalpha()]
    # store as string
    text =  ' '.join(text)

    # insert 'startseq', 'endseq' cho chuỗi
    text = 'startseq ' + text + ' endseq'
    return text

print(caption_preprocessing('chao .. ban $ hello980 it\'s a table.#'))

startseq chao ban its a table endseq


In [5]:
df['caption'] = df['caption'].apply(caption_preprocessing)

In [6]:
word_counts = {}
max_length = 0
for text in df['caption']:
  words = text.split()
  max_length = len(words) if (max_length < len(words)) else max_length
  for w in words:
    try:
      word_counts[w] +=1
    except:
        word_counts[w] = 1
print(len(word_counts))
print(max_length)

8777
37


In [7]:
# Chỉ lấy các từ xuất hiện trên 10 lần
word_count_threshold = 10
vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

preprocessed words 8777 -> 1952


In [8]:
embedding_dim = 200
vocab_size = len(vocab) + 1 # thêm 1 padding

In [9]:
from ImageDataset import ImageDataset

dataset = ImageDataset(max_length, vocab_size)

2024-04-12 15:00:04.616719: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-12 15:00:04.794780: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-12 15:00:05.523619: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.8/lib64:
2024-04-12 15:00:05.523720: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [10]:
image_shape = (len(dataset[0][0][0][0]), )

/home/dniit/ai_code/image_captioning/ImageDataset.py:50: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  return [torch.Tensor(x_image), torch.Tensor(x_cap).to(torch.int64)], torch.Tensor(y)


In [11]:
from torch.utils.data import DataLoader, Dataset

def generate_batches(dataset: Dataset,
                     batch_size: int,
                     shuffle: bool=True,
                     drop_last: bool=True) -> DataLoader:
    """"""
    return DataLoader(dataset=dataset, batch_size=batch_size,
                      shuffle=shuffle, drop_last=drop_last)


batches = generate_batches(dataset, 16)

In [12]:
from ImageCaptioningModel import ImageCaptioningModel

model = ImageCaptioningModel(image_shape=image_shape,
                             max_length=max_length,
                             vocab_size=vocab_size,
                             embedding_dim=embedding_dim)

In [13]:
model.parameters()

<generator object Module.parameters at 0x7f1d4a5f3680>

In [16]:
!export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH export

In [17]:
import torch.optim as optim
import torch.nn as nn
import torch

device = 'cuda'
model = model.to(device)
# model.load_state_dict(torch.load('weights_40epoches.h5'))
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode='min', factor=0.8,
                                                 patience=1)
loss_func = nn.CrossEntropyLoss()

In [18]:
from tqdm import tqdm

phases = ['train', 'validation']
losses = {}
accuracies = {}
num_epochs = 5
# device = 'cuda'

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-' * 10)

    for phase in phases:
        print('Phase:', phase)
        if phase == 'train':
            # scheduler.step()
            model.train()
        else:
            model.eval()
            break

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(dataset.get_batch(128)):
            inputs = [input_.to(device) for input_ in inputs]
            # inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = loss_func(outputs, labels)

            if phase == 'train':
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                # scheduler.step(loss)

            _, preds = torch.max(outputs, 1)
            _, trues = torch.max(labels, 1)
            running_loss += loss.detach() * inputs[0].size(0)
            running_corrects += torch.sum(preds == trues)

        epoch_loss = running_loss / len(dataset)
        epoch_acc = running_corrects.float() / len(dataset)

        losses[phase].append(epoch_loss.item())
        accuracies[phase].append(epoch_acc.item())

        # log_metric(f'{phase}_loss', epoch_loss, epoch)
        # log_metric(f'{phase}_accuracy', float(epoch_acc), epoch)

        print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                    epoch_loss.item(),
                                                    epoch_acc.item()))

Epoch 1/5
----------
Phase: train


1544it [1:38:32,  3.83s/it]